# Clustering 2D points via K-Means Algorithm

In [1]:
import math
import random

In [2]:
'''
Lets first define a class for representing a point 
     - an array coords consisting of coordinates in the n-d representation of the points 
     - an integer n denoting the dimension
'''

class Point(object):
    '''
    A point in n dimensional space
    '''
    def __init__(self, coords):
        '''
        coords - A list of values, one per dimension
        '''

        self.coords = coords
        self.n = len(coords)

    def __repr__(self):
        return str(self.coords)

In [3]:
p = Point([2,3])
print(p)
print(p.coords)
print(p.n)

[2, 3]
[2, 3]
2


In [4]:
def makeRandomPoint(n, lower, upper):
    '''
    Returns a Point object with n dimensions and values between lower and
    upper in each of those dimensions
    '''
    p = Point([random.uniform(lower, upper) for _ in range(n)])
    return p

makeRandomPoint(2,0,10)

[5.198895717289689, 3.3784629292787125]

In [5]:
def getDistance(a, b):
    '''
    Euclidean distance between two n-dimensional points.
    '''
    if a.n != b.n:
        raise Exception("ERROR: non comparable points")

    accumulatedDifference = 0.0
    for i in range(a.n):
        squareDifference = pow((a.coords[i]-b.coords[i]), 2)
        accumulatedDifference += squareDifference
    distance = math.sqrt(accumulatedDifference)

    return distance

In [6]:
#Assigment 1
a = [8.957, 6.936]
b = [0.189, 2.606]
getDistance(Point(a),Point(b))

9.778891757249388

In [7]:
# Assigment 2
a = makeRandomPoint(2,0,10)
b = makeRandomPoint(2,0,10)
getDistance(a,b)


2.082467069392876

In [8]:
'''
Another class for representing a point 
     - an array points of Point objects discussed above 
     - an integer n denoting the dimension of the points (assuming it to be the same for all the points)
     - a Point object centroid, defining the center of the cluster
     - a method calculateCentroid, that computes the center of the current cluster
     - a method updateCentroid, to update the centroid and return the shift seen
'''

class Cluster(object):
    '''
    A set of points and their centroid
    '''

    def __init__(self, points):
        '''
        points - A list of point objects
        '''

        if len(points) == 0:
            raise Exception("ERROR: empty cluster")

        # The points that belong to this cluster
        self.points = points

        # The dimensionality of the points in this cluster
        self.n = points[0].n

        # Assert that all points are of the same dimensionality
        for p in points:
            if p.n != self.n:
                raise Exception("ERROR: inconsistent dimensions")

        # Set up the initial centroid (this is usually based off one point)
        self.centroid = self.calculateCentroid()

    def __repr__(self):
        '''
        String representation of this object
        '''
        return str(self.points)

    def update(self, points):
        '''
        Returns the distance between the previous centroid and the new after
        recalculating and storing the new centroid.
        Note: Initially we expect centroids to shift around a lot and then
        gradually settle down.
        '''
        old_centroid = self.centroid
        self.points = points
        self.centroid = self.calculateCentroid()
        shift = getDistance(old_centroid, self.centroid)
        return shift

    def calculateCentroid(self):
        '''
        Finds a virtual center point for a group of n-dimensional points
        '''
        numPoints = len(self.points)
        # Get a list of all coordinates in this cluster
        coords = [p.coords for p in self.points]
        # Reformat that so all x's are together, all y'z etc.
        unzipped = zip(*coords)
        # Calculate the mean for each dimension
        centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]

        return Point(centroid_coords)

In [9]:
# Assigment 3 
points = [a,b]
cluster = Cluster(points)

print (cluster.points)
print (cluster.n)
print (cluster.centroid)

[[2.615767951360292, 1.8621362192246027], [0.6538667562890099, 2.560428991551247]]
2
[1.6348173538246509, 2.211282605387925]


In [10]:
# Helper method to plot a cluster
import plotly
from plotly.graph_objs import Scatter, Scatter3d, Layout
import matplotlib.pyplot as plt

def plotClusters(data, dimensions):
    '''
    This uses the plotly offline mode to create a local HTML file.
    This should open your default web browser.
    '''
    if dimensions not in [2, 3]:
        raise Exception("Plots are only available for 2 and 3 dimensional data")

    # Convert data into plotly format.
    traceList = []
    for i, c in enumerate(data):
        # Get a list of x,y coordinates for the points in this cluster.
        cluster_data = []
        for point in c.points:
            cluster_data.append(point.coords)

        trace = {}
        centroid = {}
        if dimensions == 2:
            # Convert our list of x,y's into an x list and a y list.
            trace['x'], trace['y'] = zip(*cluster_data)
            trace['mode'] = 'markers'
            trace['marker'] = {}
            trace['marker']['symbol'] = i
            trace['marker']['size'] = 12
            trace['name'] = "Cluster " + str(i)
            traceList.append(Scatter(**trace))
            # Centroid (A trace of length 1)
            centroid['x'] = [c.centroid.coords[0]]
            centroid['y'] = [c.centroid.coords[1]]
            centroid['mode'] = 'markers'
            centroid['marker'] = {}
            centroid['marker']['symbol'] = i
            centroid['marker']['color'] = 'rgb(200,10,10)'
            centroid['name'] = "Centroid " + str(i)
            traceList.append(Scatter(**centroid))
        else:
            symbols = [
                "circle",
                "square",
                "diamond",
                "circle-open",
                "square-open",
                "diamond-open",
                "cross", "x"
            ]
            symbol_count = len(symbols)
            if i > symbol_count:
                print ("Warning: Not enough marker symbols to go around")
            # Convert our list of x,y,z's separate lists.
            trace['x'], trace['y'], trace['z'] = zip(*cluster_data)
            trace['mode'] = 'markers'
            trace['marker'] = {}
            trace['marker']['symbol'] = symbols[i]
            trace['marker']['size'] = 12
            trace['name'] = "Cluster " + str(i)
            traceList.append(Scatter3d(**trace))
            # Centroid (A trace of length 1)
            centroid['x'] = [c.centroid.coords[0]]
            centroid['y'] = [c.centroid.coords[1]]
            centroid['z'] = [c.centroid.coords[2]]
            centroid['mode'] = 'markers'
            centroid['marker'] = {}
            centroid['marker']['symbol'] = symbols[i]
            centroid['marker']['color'] = 'rgb(200,10,10)'
            centroid['name'] = "Centroid " + str(i)
            traceList.append(Scatter3d(**centroid))

    title = "K-means clustering with %s clusters" % str(len(data))
    plotly.offline.plot({
        "data": traceList,
        "layout": Layout(title=title)
    })

In [11]:
# Assigment 4
plotClusters([cluster],2 )

In [12]:
# create a cluster of 10 points and plot it
points = [makeRandomPoint(2,0,10) for i in range(20)]

cluster = Cluster(points)
plotClusters([cluster],2)

In [13]:
# the function that takes in the list of points and a value k that 

def kmeans(points, k, cutoff):

    # Pick out k random points to use as our initial centroids
    initial = random.sample(points, k)

    # Create k clusters using those centroids
    # Note: Cluster takes lists, so we wrap each point in a list here.
    clusters = [Cluster([p]) for p in initial]

    # Loop through the dataset until the clusters stabilize
    loopCounter = 0
    while True:
        # Create a list of lists to hold the points in each cluster
        lists = [[] for _ in clusters]
        clusterCount = len(clusters)

        # Start counting loops
        loopCounter += 1
        # For every point in the dataset ...
        for p in points:
            # Get the distance between that point and the centroid of the first
            # cluster.
            smallest_distance = getDistance(p, clusters[0].centroid)

            # Set the cluster this point belongs to
            clusterIndex = 0

            # For the remainder of the clusters ...
            for i in range(clusterCount - 1):
                # calculate the distance of that point to each other cluster's
                # centroid.
                distance = getDistance(p, clusters[i+1].centroid) # Assignment
                # If it's closer to that cluster's centroid update what we
                # think the smallest distance is
                
                # Assigment - next 3 lines
                if distance < smallest_distance:
                    smallest_distance = distance
                    clusterIndex = i+1
                    
            # After finding the cluster the smallest distance away
            # set the point to belong to that cluster
            lists[clusterIndex].append(p)

        # Set our biggest_shift to zero for this iteration
        biggest_shift = 0.0

        # For each cluster ...
        for i in range(clusterCount):
            # Calculate how far the centroid moved in this iteration
            shift = clusters[i].update(lists[i])
            # Keep track of the largest move from all cluster centroid updates
            biggest_shift = max(biggest_shift, shift)

        # If the centroids have stopped moving much, say we're done!
        if biggest_shift < cutoff:
            print("Converged after %s iterations" % loopCounter)
            break
    return clusters

In [14]:
no_of_clusters = 3
cutoff = 0.2
clusters = kmeans(points, no_of_clusters, cutoff)

for i,c in enumerate(clusters):
    for p in c.points:
        print("cluster:" + str(i) + ", points :" + str(p))

Converged after 3 iterations
cluster:0, points :[1.2998889422242677, 6.389285669960259]
cluster:0, points :[0.037256901851570046, 8.954386962599393]
cluster:0, points :[0.563861595551407, 6.04424382895195]
cluster:0, points :[3.451806776953358, 8.560680876295638]
cluster:1, points :[9.828889633583268, 5.241660249996158]
cluster:1, points :[8.284303621671581, 7.058712167961135]
cluster:1, points :[9.238831122072842, 8.797254170229774]
cluster:1, points :[6.24210758220472, 6.97926385527949]
cluster:1, points :[6.98585710852967, 9.180951859722802]
cluster:1, points :[9.421579797391898, 9.221490795730363]
cluster:2, points :[5.7487035655118115, 2.5430131463882066]
cluster:2, points :[5.871434759923309, 3.113536840058837]
cluster:2, points :[6.671884912863371, 2.2178067488410678]
cluster:2, points :[3.680452860359419, 2.9703447850701146]
cluster:2, points :[4.505387462056067, 3.793290154086153]
cluster:2, points :[8.242851267704609, 2.3641167611954272]
cluster:2, points :[2.293195957523988,

In [17]:
plotClusters(clusters, 2)

# Play around

In [16]:
# Assigment 5